# Composition

LangChain은 프롬프트의 다양한 부분을 함께 구성하는 데 사용할 수 있는 사용자 친화적인 인터페이스를 제공한다. 이 작업은 문자열 프롬프트나 채팅 프롬프트를 사용하여 수행될 수 있다. 이런 방식으로 프롬프트를 구성함으로써, 구성 요소들의 재사용이 용이해짐을 볼 수 있다.

## String prompt composition

문자열 프롬프트를 작업할 때, 각 템플릿은 함께 연결된다. 프롬프트를 직접 작업하거나 문자열을 사용할 수 있다(리스트의 첫 번째 요소는 프롬프트여야 한다).

In [2]:
from langchain.prompts import PromptTemplate

In [3]:
prompt = (
    PromptTemplate.from_template("Tell me a joke about {topic}")
    + ", make it funny"
    + "\n\nand in {language}"
)

In [4]:
prompt

PromptTemplate(input_variables=['language', 'topic'], template='Tell me a joke about {topic}, make it funny\n\nand in {language}')

In [5]:
prompt.format(topic="sports", language="spanish")

'Tell me a joke about sports, make it funny\n\nand in spanish'

이전과 마찬가지로 LLMChain에서도 사용할 수 있다.

In [6]:
from langchain.chains import LLMChain
from langchain_openai import ChatOpenAI

In [7]:
model = ChatOpenAI()

In [8]:
chain = LLMChain(llm=model, prompt=prompt)

In [10]:
chain.run(topic="sports", language="Korean")

'왜 골프 선수는 항상 신발을 찾을까요? \n\nBecause they always have a hole-in-one! \n\n(골프 선수들은 항상 한 개만의 홀을 찾으려고 해서요!)'

## Chat prompt composition

채팅 프롬프트는 메시지의 리스트로 구성된다. 개발자 경험을 위해, 이 프롬프트를 생성하는 편리한 방법이 추가되었다. 이 파이프라인에서, 각 새로운 요소는 최종 프롬프트에서 새로운 메시지가 된다.

In [ ]:
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage

먼저, 시스템 메시지로 기본 ChatPromptTemplate를 초기화 한다. 반드시 시스템으로 시작할 필요는 없지만, 종종 좋은 관행이 된다.

In [2]:
prompt = SystemMessage(content="You are a nice pirate")

다른 메시지 또는 메시지 템플릿과 결합하여 파이프라인을 쉽게 생성할 수 있다. 변수를 포맷할 필요가 없을 때는 `Message`를 사용하고, 변수를 포맷해야 할 때는 `MessageTemplate`을 사용. 문자열만을 사용할 수도 있다(참고: 이 경우 자동으로 HumanMessagePromptTemplate으로 추론).

In [3]:
new_prompt = (
    prompt + HumanMessage(content="hi") + AIMessage(content="what?") + "{input}"
)

내부적으로, 이는 ChatPromptTemplate 클래스의 인스턴스를 생성한다. 그래서 이전과 같이 사용할 수 있다!

In [4]:
new_prompt.format_messages(input="i said hi")

[SystemMessage(content='You are a nice pirate', additional_kwargs={}),
 HumanMessage(content='hi', additional_kwargs={}, example=False),
 AIMessage(content='what?', additional_kwargs={}, example=False),
 HumanMessage(content='i said hi', additional_kwargs={}, example=False)]

또한, 이전처럼 LLMChain에서도 사용할 수 있다!

In [5]:
from langchain.chains import LLMChain
from langchain_openai import ChatOpenAI

In [6]:
model = ChatOpenAI()

In [7]:
chain = LLMChain(llm=model, prompt=new_prompt)

In [8]:
chain.run("i said hi")

'Oh, hello! How can I assist you today?'